In [77]:
from __future__ import print_function, division
import numpy as np
np.random.seed(42)
import tensorflow as tf
import pdb
import fxns

In [126]:
# data locations
raw_data_path = '../data/shuffled_examples'
outbase = '../output/shuffled_examples'
train_out_path = '%s/texttrain' % outbase
valid_out_path = '%s/textvalidate' % outbase
test_out_path = '%s/texttest' % outbase

# training hyperparameters
batch_size = 10
num_batches_in_train = int(11000 / batch_size)
num_epochs = 2
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

In [127]:
tf.reset_default_graph()

# get training data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('inputs are of size', info['seq_len'])

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('outputs are of size', info['label_len'])

inputs are of size 4000
outputs are of size 24


In [128]:
import logreg_model; reload(logreg_model)
modelname='logreg'

# get model
X, Y, loss, logits = logreg_model.get_logreg_model(info['seq_len'], info['label_len'])

# define optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,
                                                 name='SGD-Optimizer')
    update = optimizer.minimize(loss)

# define other summaries we want
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

In [130]:
if tf.gfile.Exists('log/' + modelname):
   tf.gfile.DeleteRecursively('log/' + modelname) 

# train
with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/' + modelname + '/train', sess.graph)
    writerv = tf.summary.FileWriter('log/' + modelname + '/valid')
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        average_loss = 0
        for i in range(num_batches_in_train):
            batch_X, batch_Y = sess.run([seq_batch, label_batch])
            batch_Xv, batch_Yv = sess.run([seqv_batch, labelv_batch])
            _, _loss = sess.run([update, loss],
                            feed_dict={X: batch_X, Y: batch_Y})
            average_loss += _loss / num_batches_in_train
            
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            summaryv = sess.run(summary_op, feed_dict={X: batch_Xv,
                                                    Y: batch_Yv})
            writer.add_summary(summary, global_step=epoch*num_batches_in_train + i)
            writerv.add_summary(summaryv, global_step=epoch*num_batches_in_train + i)
        print('Epoch:',epoch,'Avg loss:',average_loss)
    
    writer.close()
    coord.request_stop()
    coord.join(threads)

Epoch: 0 Avg loss: 0.192156968279
Epoch: 1 Avg loss: 0.168483015204
